In [ ]:
import geopandas as gpd
import numpy as np
import os
from shapely.geometry import Polygon
from math import sin, cos, asin, atan, sqrt, pow, radians, degrees
print(os.getcwd())


In [ ]:
def distance_matrix(x0, y0, x1, y1):
    obs = np.vstack((x0, y0)).T
    interp = np.vstack((x1, y1)).T

    # Make a distance matrix between pairwise observations
    # Note: from <http://stackoverflow.com/questions/1871536>
    # (Yay for ufuncs!)
    d0 = np.subtract.outer(obs[:,0], interp[:,0])
    d1 = np.subtract.outer(obs[:,1], interp[:,1])

    return np.hypot(d0, d1)

def ddd2dircos(dip,dipdir):
    l = sin(radians(dipdir))*cos(radians(90-dip))
    m = cos(radians(dipdir))*cos(radians(90-dip))
    n = sin(radians(90-dip))
    return(l,m,n)

def dircos2ddd(l,m,n):
    if(m>0):
        dipdir=(360+degrees(atan(l/m)))%360
    else:
        dipdir=(540+degrees(atan(l/m)))%360
    dip =90-degrees(asin(n))
    return(dip,dipdir)

In [ ]:
geology_file='./DataForVariance/Lithos_2019.shp'
structure_file='./DataForVariance/AllStructures_Fall2019_Merged_Corrected.shp'

In [ ]:
minx=518000    #left hams
maxx=553000    #right
miny=5474000   #bottom
maxy=5494000   #top
src_crs = 'epsg:4326'  #input data coord ref system (assumed to be geodetic lat/long WGS84)
dst_crs = 'epsg:32615' #model ref system (assumed to be something with metre coordinates)

buffer=2000

bbox=str(miny)+","+str(minx)+","+str(maxy)+","+str(maxx)
lat_point_list = [miny, miny, maxy, maxy, maxy]
lon_point_list = [minx, maxx, maxx, minx, minx]

bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom]) 


In [ ]:
geology = gpd.read_file(geology_file)
structure = gpd.read_file(structure_file)



In [ ]:
base=geology.plot(column='LABELID',figsize=(10,10),edgecolor='#000000',linewidth=0.1)
polygon.plot(ax=base, color='none',edgecolor='black')
structure.plot(ax=base, color='none',edgecolor='black')

In [ ]:
xi = np.linspace(minx,maxx,35)
yi = np.linspace(miny,maxy,20)

xi, yi = np.meshgrid(xi, yi)
xi, yi = xi.flatten(), yi.flatten()


In [ ]:
display(len(structure))
x = np.zeros(len(structure))
y = np.zeros(len(structure))
dip = np.zeros(len(structure))
azimuth = np.zeros(len(structure))






In [ ]:
npt=0
for a_pt in structure.iterrows():
    if('BED' in a_pt[1]['OGS_symbol']  ):
        x[npt]=float(a_pt[1].geometry.x)
        y[npt]=float(a_pt[1].geometry.y)
        dip[npt]=float(a_pt[1]['dip'])
        azimuth[npt]=float(a_pt[1]['azimuth'])
        npt=npt+1
    

In [ ]:
display(npt)

In [ ]:
dist= distance_matrix(x[0:npt],y[0:npt], xi,yi)

In [ ]:
display(dist)
display(dist.shape)

In [ ]:
display(xi[0],yi[0])

In [ ]:
f=open('output/pt_density.csv','w')
f.write('x,y,npts\n')
for i in range(0,len(xi)):
    a_col=dist[:,i:i+1]
    is_close=a_col<buffer 
    close=a_col[is_close]
    ostr=str(xi[i])+','+str(yi[i])+','+str(len(close))+'\n'
    f.write(ostr)
f.close()        

In [ ]:
#dips=structure['dip']
#azimuths=structure['azimuth']

In [ ]:
display(dip[0:npt],dip[0:npt].shape)

l = np.zeros(npt)
m = np.zeros(npt)
n = np.zeros(npt)

In [ ]:
dip=dip[0:npt].reshape(npt,1)
azimuth=azimuth[0:npt].reshape(npt,1)

In [ ]:
f=open('output/pt_mean_orientation.csv','w')
f.write('x,y,dip,azimuth\n')
for i in range(0,len(xi)):
    a_dist=dist[:,i:i+1]
    is_close=a_dist<buffer 
    close_dip=dip[is_close]
    close_azimuth=azimuth[is_close]
    n_good=0
    for j in range(0,len(close_dip)):
        if(close_dip[j]>=0.0 and close_dip[j]<=90.0 
           and close_azimuth[j] >= 0.0 and close_azimuth[j] <= 360.0):
            l[n_good],m[n_good],n[n_good]=ddd2dircos(float(close_dip[j]),float(close_azimuth[j])+90.0)
            n_good=n_good+1
    if(n_good>0):
        lm=np.mean(l[:n_good])
        mm=np.mean(m[:n_good])
        nm=np.mean(n[:n_good])
        #print(lm,mm,nm)
        dip_mean,dipdirection_mean=dircos2ddd(lm,mm,nm)
        ostr=str(xi[i])+','+str(yi[i])+','+str(dip_mean)+','+str(dipdirection_mean-90)+'\n'
        f.write(ostr)
f.close()        